#Sentence Embeddings - Metacritics Sentiment Analysis
10 fold Cross Validation
## [Panda](https://github.com/PANDA-UFSCar) - 2023/2
Autora: [Letícia Bossato Marchezi](linkedin.com/in/letmarchezi/)

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive', force_remount=True)
%cd "gdrive/MyDrive/Grupo 1 - Processamento de Linguagem Natural/Data/"

Mounted at /content/gdrive
/content/gdrive/.shortcut-targets-by-id/1ub11KA5pjUO4RCNqv5VFfBB4Tnv21ooN/Grupo 1 - Processamento de Linguagem Natural/Data


In [ ]:
import pandas as pd
df = pd.read_csv("preprocessed-metacritics-total.csv")
df.head()

,Movie name,Review,Created at,Score,Genre
0,Arrival,"['denis', 'villeneuve', 'shows', 'us', 'all', ...","OCT 3, 2022",1.0,Mistery
1,Arrival,"['amy', 'adams', 'gives', 'a', 'superb', 'perf...","MAR 7, 2022",1.0,Mistery
2,Arrival,"['this', 'movie', 'is', 'not', 'for', 'everyon...","DEC 6, 2019",1.0,Mistery
3,Arrival,"['arrival', 'is', 'one', 'of', 'my', 'favorite...","APR 3, 2020",1.0,Mistery
4,Arrival,"['i', 'do', 'not', 'think', 'this', 'movie', '...","MAR 2, 2020",1.0,Mistery


In [ ]:
import numpy as np
np.random.seed(42)

In [ ]:
!pip install sentence-transformers spacy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 3.1 MB/s eta 0:00:00


**Baixando o modelo pré-treinado de sentence embeddings.**
O modelo 'all-MiniLM-L6-v2' possui 384 dimensões e foi treinado com mais de 1 bilhão de pares de sentenças.

Referência: https://www.sbert.net
https://www.sbert.net

In [ ]:
from sentence_transformers import SentenceTransformer
from tqdm.notebook import tqdm
model = SentenceTransformer('all-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
!python -m spacy download en_core_web_sm

2024-01-30 19:06:41.923900: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-30 19:06:41.924004: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-30 19:06:41.926547: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-30 19:06:45.178391: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 17.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


## Visualização e pré-processamento
Outros passos de pré-processamento como remoção de símbolos especiais e sinais de pontuação já foram removidos, porém serão eliminadas também as stopwords

In [ ]:
import spacy
import string
nlp = spacy.load("en_core_web_sm")
stop_words = nlp.Defaults.stop_words
print(stop_words)

{"'d", 'wherein', 'many', 'though', 'namely', 'up', 'somehow', 'yours', 'who', 'by', 'one', 'ours', 'for', 'however', 'empty', 'quite', 'becomes', 'through', 'the', 'due', 'doing', 'get', 'onto', 'into', 'bottom', 'do', 'this', 'using', 'everyone', 'name', 'others', 'our', 'whole', 'further', 're', 'seemed', 'regarding', 'anything', 'thus', 'else', 'somewhere', 'myself', 'neither', 'will', 'had', 'along', 'we', 'none', 'serious', 'can', 'rather', 'herein', 'than', 'out', 'sometime', 'my', 'everything', 'towards', 'should', 'because', 'at', 'all', 'which', 'were', 'even', 'you', 'of', 'together', 'amongst', 'nothing', 'also', 'cannot', "'m", '‘m', 'beside', 'first', 'well', "'ve", 'ourselves', 'n‘t', '‘ll', 'yourselves', 'after', 'toward', '‘d', 'when', 'here', 'he', 'seem', 'latterly', 'just', 'formerly', '’m', 'but', 'them', 'us', 'upon', 'from', "'re", 'next', 'otherwise', 'itself', 'is', 'their', 'beforehand', 'take', 'always', 'something', 'hereupon', '’ll', 'six', "n't", 'top', 'b

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6480 entries, 0 to 6479
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Movie name  6480 non-null   object 
 1   Review      6480 non-null   object 
 2   Created at  6480 non-null   object 
 3   Score       6480 non-null   float64
 4   Genre       6480 non-null   object 
dtypes: float64(1), object(4)
memory usage: 253.2+ KB


In [ ]:
df.Genre.value_counts()

Mistery      540
Family       540
Horror       540
Fantasy      540
Drama        540
Thriller     540
sci-fi       540
sport        540
comedy       540
Crime        540
Adventure    540
Animation    540
Name: Genre, dtype: int64

In [ ]:
df.Score.value_counts()

 1.0    2160
 0.0    2160
-1.0    2160
Name: Score, dtype: int64

In [ ]:
df.head()

,Movie name,Review,Created at,Score,Genre
0,Arrival,"['denis', 'villeneuve', 'shows', 'us', 'all', ...","OCT 3, 2022",1.0,Mistery
1,Arrival,"['amy', 'adams', 'gives', 'a', 'superb', 'perf...","MAR 7, 2022",1.0,Mistery
2,Arrival,"['this', 'movie', 'is', 'not', 'for', 'everyon...","DEC 6, 2019",1.0,Mistery
3,Arrival,"['arrival', 'is', 'one', 'of', 'my', 'favorite...","APR 3, 2020",1.0,Mistery
4,Arrival,"['i', 'do', 'not', 'think', 'this', 'movie', '...","MAR 2, 2020",1.0,Mistery


### Conversão para lista de tokens

In [ ]:
import ast

df['Review'] = df['Review'].apply(ast.literal_eval)

In [ ]:
df['Review'].head()

0    [denis, villeneuve, shows, us, all, his, talen...
1    [amy, adams, gives, a, superb, performance, in...
2    [this, movie, is, not, for, everyone, if, you,...
3    [arrival, is, one, of, my, favorite, sci, fi, ...
4    [i, do, not, think, this, movie, is, about, th...
Name: Review, dtype: object

In [ ]:
df["String"] = df["Review"].apply(lambda x: ' '.join(x))

In [ ]:
df.head()

,Movie name,Review,Created at,Score,Genre,String
0,Arrival,"[denis, villeneuve, shows, us, all, his, talen...","OCT 3, 2022",1.0,Mistery,denis villeneuve shows us all his talent in th...
1,Arrival,"[amy, adams, gives, a, superb, performance, in...","MAR 7, 2022",1.0,Mistery,amy adams gives a superb performance in what c...
2,Arrival,"[this, movie, is, not, for, everyone, if, you,...","DEC 6, 2019",1.0,Mistery,this movie is not for everyone if you wanted a...
3,Arrival,"[arrival, is, one, of, my, favorite, sci, fi, ...","APR 3, 2020",1.0,Mistery,arrival is one of my favorite sci fi movies of...
4,Arrival,"[i, do, not, think, this, movie, is, about, th...","MAR 2, 2020",1.0,Mistery,i do not think this movie is about that what m...


### Remoção de stopwords

In [ ]:
def spacy_tokenizer(sentence):
    # Process the sentence using spaCy
    doc = nlp(sentence)

    # Lemmatize, lowercase, strip and remove stop words in one go
    filtered_sentence = " ".join([token.lemma_.lower() for token in doc if token.lemma_.lower() not in stop_words])
    return filtered_sentence

Exemplo de tokenização

In [ ]:
print(f"Texto original: \n{(df['String'][0])}")
print(f"Pré-processamento com spacy: \n{spacy_tokenizer(df['String'][0])}")

Texto original: 
denis villeneuve shows us all his talent in this film which is for me his best science fiction is used in the best way i have rarely been so taken by the mystery of a film during my first cinema session
Pré-processamento com spacy: 
denis villeneuve talent film good science fiction use good way rarely mystery film cinema session


In [ ]:
df.head()

,Movie name,Review,Created at,Score,Genre,String
0,Arrival,"[denis, villeneuve, shows, us, all, his, talen...","OCT 3, 2022",1.0,Mistery,denis villeneuve shows us all his talent in th...
1,Arrival,"[amy, adams, gives, a, superb, performance, in...","MAR 7, 2022",1.0,Mistery,amy adams gives a superb performance in what c...
2,Arrival,"[this, movie, is, not, for, everyone, if, you,...","DEC 6, 2019",1.0,Mistery,this movie is not for everyone if you wanted a...
3,Arrival,"[arrival, is, one, of, my, favorite, sci, fi, ...","APR 3, 2020",1.0,Mistery,arrival is one of my favorite sci fi movies of...
4,Arrival,"[i, do, not, think, this, movie, is, about, th...","MAR 2, 2020",1.0,Mistery,i do not think this movie is about that what m...


In [ ]:
%%time
df["No_stopwords"] = df["String"].apply(lambda x: spacy_tokenizer(x))

CPU times: user 2min 33s, sys: 347 ms, total: 2min 34s
Wall time: 2min 40s


In [ ]:
df.head()

,Movie name,Review,Created at,Score,Genre,String,No_stopwords
0,Arrival,"[denis, villeneuve, shows, us, all, his, talen...","OCT 3, 2022",1.0,Mistery,denis villeneuve shows us all his talent in th...,denis villeneuve talent film good science fict...
1,Arrival,"[amy, adams, gives, a, superb, performance, in...","MAR 7, 2022",1.0,Mistery,amy adams gives a superb performance in what c...,amy adam superb performance independence day b...
2,Arrival,"[this, movie, is, not, for, everyone, if, you,...","DEC 6, 2019",1.0,Mistery,this movie is not for everyone if you wanted a...,movie want science fiction action blockbuster ...
3,Arrival,"[arrival, is, one, of, my, favorite, sci, fi, ...","APR 3, 2020",1.0,Mistery,arrival is one of my favorite sci fi movies of...,arrival favorite sci fi movie time movie need ...
4,Arrival,"[i, do, not, think, this, movie, is, about, th...","MAR 2, 2020",1.0,Mistery,i do not think this movie is about that what m...,think movie human personally think create real...


## Encoding dos embeddings
Conversão dos dados não estruturados (texto) para dados numéricos

In [ ]:
%%time
df['embeddings'] = df['No_stopwords'].apply(model.encode)

CPU times: user 4min 41s, sys: 648 ms, total: 4min 42s
Wall time: 4min 46s


In [ ]:
df.head()

,Movie name,Review,Created at,Score,Genre,String,No_stopwords,embeddings
0,Arrival,"[denis, villeneuve, shows, us, all, his, talen...","OCT 3, 2022",1.0,Mistery,denis villeneuve shows us all his talent in th...,denis villeneuve talent film good science fict...,"[-0.051069424, -0.03442476, 0.040604454, 0.003..."
1,Arrival,"[amy, adams, gives, a, superb, performance, in...","MAR 7, 2022",1.0,Mistery,amy adams gives a superb performance in what c...,amy adam superb performance independence day b...,"[-0.019002363, -0.084105626, 0.05297816, 0.020..."
2,Arrival,"[this, movie, is, not, for, everyone, if, you,...","DEC 6, 2019",1.0,Mistery,this movie is not for everyone if you wanted a...,movie want science fiction action blockbuster ...,"[-0.12560546, -0.04636644, 0.0021197349, -0.00..."
3,Arrival,"[arrival, is, one, of, my, favorite, sci, fi, ...","APR 3, 2020",1.0,Mistery,arrival is one of my favorite sci fi movies of...,arrival favorite sci fi movie time movie need ...,"[-0.07113165, -0.06138296, 0.035424523, 0.0222..."
4,Arrival,"[i, do, not, think, this, movie, is, about, th...","MAR 2, 2020",1.0,Mistery,i do not think this movie is about that what m...,think movie human personally think create real...,"[-0.011855232, -0.033878442, -0.04039146, -0.0..."


In [ ]:
df.to_csv("../Scripts/Classificação/Embeddings/sentence-embeddings-corpus.csv",index=False)

In [ ]:
#df = pd.read_csv("../Scripts/Classificação/Embeddings/sentence-embeddings-corpus.csv")

In [ ]:
df.head()

,Movie name,Review,Created at,Score,Genre,String,No_stopwords,embeddings
0,Arrival,"['denis', 'villeneuve', 'shows', 'us', 'all', ...","OCT 3, 2022",1.0,Mistery,denis villeneuve shows us all his talent in th...,denis villeneuve talent film good science fict...,[-5.10694236e-02 -3.44247594e-02 4.06044535e-...
1,Arrival,"['amy', 'adams', 'gives', 'a', 'superb', 'perf...","MAR 7, 2022",1.0,Mistery,amy adams gives a superb performance in what c...,amy adam superb performance independence day b...,[-1.90023631e-02 -8.41056257e-02 5.29781617e-...
2,Arrival,"['this', 'movie', 'is', 'not', 'for', 'everyon...","DEC 6, 2019",1.0,Mistery,this movie is not for everyone if you wanted a...,movie want science fiction action blockbuster ...,[-1.25605464e-01 -4.63664383e-02 2.11973488e-...
3,Arrival,"['arrival', 'is', 'one', 'of', 'my', 'favorite...","APR 3, 2020",1.0,Mistery,arrival is one of my favorite sci fi movies of...,arrival favorite sci fi movie time movie need ...,[-7.11316466e-02 -6.13829605e-02 3.54245231e-...
4,Arrival,"['i', 'do', 'not', 'think', 'this', 'movie', '...","MAR 2, 2020",1.0,Mistery,i do not think this movie is about that what m...,think movie human personally think create real...,[-1.18552316e-02 -3.38784419e-02 -4.03914601e-...


In [ ]:
X = df['embeddings'].to_list()
y = df['Score'].to_list()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import numpy as np  # Import NumPy for array operations
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import StratifiedKFold

## 10 Fold Cross Validation

In [ ]:
def custom_cross_val_metrics(classifier, X, y, k=10,verbose=False,standarlization=False):
    """
    Perform k-fold cross-validation and calculate metrics for each fold.

    Parameters:
    - classifier: The classifier to evaluate.
    - X: The feature matrix.
    - y: The target labels.
    - k: The number of folds for cross-validation.

    Returns:
    - List of dictionaries containing metrics for each fold.
    """
    metrics_list = []

    skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

    # Convert X and y to NumPy arrays
    X = np.array(X)
    y = np.array(y)

    for train_index, test_index in skf.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        if standarlization:
            scaler = StandardScaler()
            X_train = scaler.fit_transform(X_train)
            X_test = scaler.transform(X_test)

        # Fit the classifier on the training data
        classifier.fit(X_train, y_train)

        # Make predictions on the test data
        y_pred = classifier.predict(X_test)

        # Calculate metrics
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred, average='macro')
        recall = recall_score(y_test, y_pred, average='macro')
        f1 = f1_score(y_test, y_pred, average='macro')

        metrics_dict = {
            'Accuracy': accuracy,
            'Precision': precision,
            'Recall': recall,
            'F1-Score': f1
        }

        metrics_list.append(metrics_dict)
        if(verbose):
          print(f"Classifier: {classifier}")
          # Print metrics for each fold
          for i, metrics in enumerate(metrics_list):
              print(f"Fold {i+1} Metrics:")
              for metric, value in metrics.items():
                  print(f"{metric}: {value}")
              print()
    return metrics_list

In [ ]:
def calc_mean_metrics(metrics_list_dic):
  # Initialize dictionaries to store the sum of each metric
  mean_metrics = {'Accuracy': 0, 'Precision': 0, 'Recall': 0, 'F1-Score': 0}

  # Calculate the sum of each metric
  for metrics_dict in metrics_list_dic:
      for metric, value in metrics_dict.items():
          mean_metrics[metric] += value

  # Calculate the mean of each metric
  num_metrics = len(metrics_list_dic)
  mean_metrics = {metric: mean_metrics[metric] / num_metrics for metric in mean_metrics}

  # Print the mean of each metric
  for metric, value in mean_metrics.items():
      print(f"Mean {metric}: {value}")
  return mean_metrics

## Treinamento e resultados

In [ ]:
#-----------------------RF-------------------------------
print("Random Forest:")
print(f"\nSem standart scaler:")
metrics_list = custom_cross_val_metrics(RandomForestClassifier(), X, y, k=10,verbose=False,standarlization=False)
mean_metric_rf = calc_mean_metrics(metrics_list)

metrics_list = custom_cross_val_metrics(RandomForestClassifier(), X, y, k=10,verbose=False,standarlization=True)
print(f"\nCom standart scaler:")
mean_metric_rf = calc_mean_metrics(metrics_list)
print("-------------------------------------------\n")
#-----------------------KNN-------------------------------
print("KNN:")
metrics_list = custom_cross_val_metrics(KNeighborsClassifier(), X, y, k=10,verbose=False,standarlization=False)
print(f"\nSem: standart scaler:")
mean_metric_knn = calc_mean_metrics(metrics_list)

metrics_list = custom_cross_val_metrics(KNeighborsClassifier(), X, y, k=10,verbose=False,standarlization=True)
print(f"\nCom standart scaler:")
mean_metric_knn = calc_mean_metrics(metrics_list)
print("-------------------------------------------\n")
#-----------------------SVM-------------------------------
print("SVM:")
metrics_list = custom_cross_val_metrics(SVC(), X, y, k=10,verbose=False,standarlization=False)
print(f"\nSem: standart scaler:")
mean_metric_svm = calc_mean_metrics(metrics_list)

metrics_list = custom_cross_val_metrics(SVC(), X, y, k=10,verbose=False,standarlization=True)
print(f"\nCom standart scaler:")
mean_metric_svm = calc_mean_metrics(metrics_list)

print("-------------------------------------------\n")
#-----------------------NB-------------------------------
print("GaussianNB:")
metrics_list = custom_cross_val_metrics(GaussianNB(), X, y, k=10,verbose=False,standarlization=False)
print(f"\nSem: standart scaler:")
mean_metric_nb = calc_mean_metrics(metrics_list)

metrics_list = custom_cross_val_metrics(GaussianNB(), X, y, k=10,verbose=False,standarlization=True)
print(f"\nCom standart scaler:")
mean_metric_nb = calc_mean_metrics(metrics_list)

print("-------------------------------------------\n")

Random Forest:

Sem standart scaler:
Mean Accuracy: 0.5695987654320988
Mean Precision: 0.5649167921199835
Mean Recall: 0.5695987654320988
Mean F1-Score: 0.5660603269888062

Com standart scaler:
Mean Accuracy: 0.5762345679012346
Mean Precision: 0.5731850113997947
Mean Recall: 0.5762345679012346
Mean F1-Score: 0.5737097263506455
-------------------------------------------

KNN:

Sem: standart scaler:
Mean Accuracy: 0.496141975308642
Mean Precision: 0.5116894748801649
Mean Recall: 0.496141975308642
Mean F1-Score: 0.4963455502589107

Com standart scaler:
Mean Accuracy: 0.4955246913580247
Mean Precision: 0.5107057014893522
Mean Recall: 0.4955246913580247
Mean F1-Score: 0.49552716375597355
-------------------------------------------

SVM:

Sem: standart scaler:
Mean Accuracy: 0.6128086419753086
Mean Precision: 0.6116089696002063
Mean Recall: 0.6128086419753086
Mean F1-Score: 0.6116295477932561

Com standart scaler:
Mean Accuracy: 0.6138888888888889
Mean Precision: 0.6129015353870501
Mean Rec

Por fim, pode-se concluir que o melhor modelo foi o Support Vector Machine com métricas superiores.